# Z-ДНК и G-квадруплексы
  
*(z-днк, казалось бы, причем тут украина...)*

## Задание
Идете на сайт UCSC genome browser,  Downloads -> Genome Data
https://hgdownload.soe.ucsc.edu/downloads.html

Выбрать любой организм КРОМЕ человека и мыши. Любой из вкладок Mammals, Other Vertebrates, Deuterostomes, Insects, Nematodes, Othergenomes. У организма достаточно взять какую-нибудь одну хромосому.

Внести его в таблицу вместе с ФИО и группой
https://docs.google.com/spreadsheets/d/1yDismD3oP7yKB-cVItI8SHdunFi-qB9w6tJutY72UD8/edit?usp=sharing

Найти в геноме участки Z-ДНК (с помощью программы Z-Hunt) и квадруплексы (c помощью паттерна, см семинар)

В отчете ответить на вопросы:
1. Сколько участков Z-ДНК найдено в геноме?
2. Сколько квадруплексов найдено в геноме?
3. В какие участки генома (гены или межгеннное пространство или на пересечении) попадают Z-ДНК и квадруплексы. Аннотацию генома генами для выбранного организма скачать из TableBrowser -  Genes and Gene Prediction Track – GeneCode annotation tracks.
4. Предоставить список генов, в промоторы которых попадает Z-ДНК. Промоторами считать участки от начала генов и upstream 1000 bp. При этом для генов на положительном и отрицательном стренде по-разному рассчитываются координаты промоторов. Для положительного стренда, если начало гена на координате 5000 bp, промоторный участок будет 4000-5000, а для отрицательного стренда 5000-6000. Имена вытаскиваются из предпоследней колонки таблицы.
5. Предоставить список генов, в промоторы которых попадает квадруплексы.
6. На 10 баллов – сделать Gene Enrichment Analysis – зайти на сайт stringDB https://string-db.org/. Нажать на Search. Выбрать слева Multiple Proteins - Найти нужный организм, в окно загрузить список генов и нажать Search. На вкладе Analysis будут результаты обогощения отобранных генов биологическими процессами. Сделать скриншот анализа. Выгрузить таблицу в Excel. 

Была выбрана хромосома 1 (`chr1`) из генома организма `Takifugu rubripes` (`fugu5/fr3`) (см. https://hgdownload.soe.ucsc.edu/goldenPath/fr3/bigZips/)  
Так как на сайте нет возможности скачать последовательности хромосом по отдельности, был скачан файл с полной геномной последовательностью в формате `2bit` (см. https://hgdownload.soe.ucsc.edu/goldenPath/fr3/bigZips/fr3.2bit).  
Далее для извлечения последовательности нужной хромосомы в формате `fasta` использовалась программа `twoBitToFa`, предоставляемая `UCSC` (см. http://hgdownload.cse.ucsc.edu/admin/exe/linux.x86_64/):

    ./twoBitToFa fr3.2bit fugu-chr1.fasta -seq=chr1

Для нахождения участков `Z-ДНК` будем использовать программу `zhunt3` (см. `./zhunt3-alan.c`). Соберем программу и запустим с параметрами из примера:

    gcc zhunt3-alan.c -lm -o zhunt3
    ./zhunt3 12 8 12 fugu-chr1.fasta

Получили файл `fugu-chr1.fasta.Z-SCORE`, содержащий `z-score` для участков из последовательности.  
Найдем участки `Z-ДНК`, используя порог по `z-score`, равный 300.

In [1]:
"""
pandas почему-то вылетает на такой таблице :(
Поэтому вручную распарсим файл по-строчно. 
"""
z_score_filepath = "./internal/fugu-chr1.fasta.Z-SCORE"
z_score_threshold = 300
z_regions = []

with open(z_score_filepath, "r") as f:
    # Пропускаем заголовок
    f.readline()

    while line := f.readline():
        line = line.split()
        z_score = float(line[5])
        
        if z_score >= z_score_threshold:
            start = int(line[0])
            end = int(line[1])
            z_regions.append((start, end))

1\. Сколько участков Z-ДНК найдено в геноме? 

In [2]:
print(len(z_regions))

280979


2\. Сколько квадруплексов найдено в геноме?

In [3]:
import re 
from Bio import SeqIO

In [4]:
sequence_filepath = "./internal/fugu-chr1.fasta"
pattern_plus_strand = "(?:G{3,}[ATGC]{1,7}){3,}G{3,}"
pattern_minus_strand = "(?:C{3,}[ATGC]{1,7}){3,}C{3,}"

sequence = str(next(SeqIO.parse(sequence_filepath, "fasta")).seq)

In [5]:
PQS_plus = [(m.start(), m.end(), m.group(0)) for m in re.finditer(pattern_plus_strand, sequence, re.IGNORECASE)]
print(f"Квадруплексов на плюс-стренде: {len(PQS_plus)}")

Квадруплексов на плюс-стренде: 1407


In [6]:
PQS_minus = [(m.start(), m.end(), m.group(0)) for m in re.finditer(pattern_minus_strand, sequence, re.IGNORECASE)]
print(f"Квадруплексов на минус-стренде: {len(PQS_minus)}")

Квадруплексов на минус-стренде: 1398


In [7]:
print(f"Квадруплексов всего: {len(PQS_plus) + len(PQS_minus)}")

Квадруплексов всего: 2805


3\. В какие участки генома (гены или межгенное пространство или на пересечении) попадают Z-ДНК и квадруплексы. Аннотацию генома генами для выбранного организма скачать из TableBrowser - Genes and Gene Prediction Track – GeneCode annotation tracks.

In [8]:
import pandas as pd

In [9]:
annotation_filepath = "./internal/fugu-chr1-annotation.tsv"

df = pd.read_csv(annotation_filepath, sep='\t')
df.head()

,#bin,name,chrom,strand,txStart,txEnd,cdsStart,cdsEnd,exonCount,exonStarts,exonEnds,score,name2,cdsStartStat,cdsEndStat,exonFrames
0,585,XR_964528.1,chr1,+,26235,26835,26835,26835,3,"26235,26480,26728,","26359,26651,26835,",0,LOC105416383,none,none,"-1,-1,-1,"
1,585,XR_964519.1,chr1,+,55374,57235,57235,57235,3,"55374,56151,56944,","55484,56410,57235,",0,LOC105416374,none,none,"-1,-1,-1,"
2,73,XM_011603754.1,chr1,+,115173,255908,115176,255908,44,"115173,218682,223433,224385,229620,229888,2300...","115237,218842,223540,224533,229754,230000,2302...",0,LOC101068579,cmpl,cmpl,"0,1,2,1,2,1,2,1,1,1,0,1,2,2,1,1,2,1,0,1,1,1,0,..."
3,587,XM_011603350.1,chr1,+,272430,274255,272603,273601,2,"272430,272732,","272730,274255,",0,LOC101064527,cmpl,cmpl,"0,1,"
4,587,XM_003961015.1,chr1,+,285687,288576,285687,288576,7,"285687,286669,286841,286999,287306,288114,288359,","286257,286753,286912,287143,287529,288269,288576,",0,tekt3,cmpl,cmpl,"0,0,0,2,2,0,2,"


In [10]:
tx_regions = []
for start, end, strand, name in zip(df["txStart"], df["txEnd"], df["strand"], df["name2"]):
    start = int(start)
    end = int(end)
    elem = {"start": start, "end": end, "strand": strand, "name": name}
    if elem not in tx_regions:
        tx_regions.append(elem)

In [11]:
def count_in_tx(regions: list[tuple[int, int]], strand: str = None) -> int: 
    count = 0

    for start, end in regions:
        for tx_region in tx_regions:
            tx_start, tx_end, tx_strand = tx_region["start"], tx_region["end"], tx_region["strand"]
            if strand is not None and tx_strand != strand:
                continue
            if tx_start > start:
                break
            if end <= tx_end:
                count += 1
                break

    return count

In [12]:
z_regions_in_tx_count = count_in_tx(z_regions)
z_regions_out_of_tx_count = len(z_regions) - z_regions_in_tx_count

print(f"Z-ДНК, попадающие в гены: {z_regions_in_tx_count}")
print(f"Z-ДНК, попадающие в межгенное пространство: {z_regions_out_of_tx_count}")

Z-ДНК, попадающие в гены: 152536
Z-ДНК, попадающие в межгенное пространство: 128443


Как видно, Z-ДНК попадает как в гены, так и в межгенное пространство

In [13]:
pqs_plus_regions = list(map(lambda pqs: (pqs[0], pqs[1]), PQS_plus))
pqs_minus_regions = list(map(lambda pqs: (pqs[0], pqs[1]), PQS_minus))

In [14]:
pqs_plus_regions_in_tx_count = count_in_tx(pqs_plus_regions, strand = "+")
pqs_plus_regions_out_of_tx_count = len(PQS_plus) - pqs_plus_regions_in_tx_count

print(f"PQS на плюс-стренде, попадающие в гены: {pqs_plus_regions_in_tx_count}")
print(f"PQS на плюс-стренде, попадающие в межгенное пространство: {pqs_plus_regions_out_of_tx_count}")

PQS на плюс-стренде, попадающие в гены: 375
PQS на плюс-стренде, попадающие в межгенное пространство: 1032


In [15]:
pqs_minus_regions_in_tx_count = count_in_tx(pqs_minus_regions, strand = "-")
pqs_minus_regions_out_of_tx_count = len(PQS_minus) - pqs_minus_regions_in_tx_count

print(f"PQS на минус-стренде, попадающие в гены: {pqs_minus_regions_in_tx_count}")
print(f"PQS на минус-стренде, попадающие в межгенное пространство: {pqs_minus_regions_out_of_tx_count}")

PQS на минус-стренде, попадающие в гены: 290
PQS на минус-стренде, попадающие в межгенное пространство: 1108


PQS попадают как в гены, так и в межгенное пространство

4\. Предоставить список генов, в промоторы которых попадает Z-ДНК.  
Промоторами считать участки от начала генов и upstream 1000 bp. При этом для генов на положительном и отрицательном стренде по-разному рассчитываются координаты промоторов. Для положительного стренда, если начало гена на координате 5000 bp, промоторный участок будет 4000-5000, а для отрицательного стренда 5000-6000.

In [16]:
upstream_bp = 1000

def get_promoted_genes(regions: list[tuple[int, int]], strand: str = None) -> list[str]:
    result = []

    for start, end in regions:
        skip_plus_strand = False
        skip_minus_strand = False

        for tx_region in tx_regions:
            tx_start, tx_strand, tx_name = tx_region["start"], tx_region["strand"], tx_region["name"]

            if strand is not None and tx_strand != strand:
                continue

            tx_promoted_start = max(0, tx_start - upstream_bp) if tx_strand == "+" else tx_start
            tx_promoted_end = tx_start if tx_strand == "+" else tx_start + upstream_bp 

            if tx_promoted_start > start:
                if tx_strand == "+":
                    skip_plus_strand = True
                else:
                    skip_minus_strand = True
                
                if skip_plus_strand and skip_minus_strand:
                    break

            if end <= tx_promoted_end:
                if tx_name not in result:
                    result.append(tx_name)
                break

    return result

In [17]:
z_promoted_genes = sorted(get_promoted_genes(z_regions))

In [18]:
z_promoted_filepath = "./files/z-promoted.txt"

with open(z_promoted_filepath, "w") as f:
    for gene in z_promoted_genes:
        f.write(gene + '\n')

Список генов, в промоторы которых попадает Z-ДНК, представлен в `./files/z-promoted.txt`

5\. Предоставить список генов, в промоторы которых попадают квадруплексы.

In [19]:
pqs_plus_promoted_genes = sorted(get_promoted_genes(pqs_plus_regions, strand = "+"))

In [20]:
pqs_plus_promoted_filepath = "./files/pqs-plus-promoted.txt"

with open(pqs_plus_promoted_filepath, "w") as f:
    for gene in pqs_plus_promoted_genes:
        f.write(gene + '\n')

In [21]:
pqs_minus_promoted_genes = sorted(get_promoted_genes(pqs_minus_regions, strand = "-"))

In [22]:
pqs_minus_promoted_filepath = "./files/pqs-minus-promoted.txt"

with open(pqs_minus_promoted_filepath, "w") as f:
    for gene in pqs_minus_promoted_genes:
        f.write(gene + '\n')

In [23]:
pqs_united_promoted_genes = pqs_plus_promoted_genes + pqs_minus_promoted_genes
pqs_united_promoted_genes.sort()

In [24]:
pqs_united_promoted_filepath = "./files/pqs-united-promoted.txt"

with open(pqs_united_promoted_filepath, "w") as f:
    for gene in pqs_united_promoted_genes:
        f.write(gene + '\n')

Список генов на плюс-стренде, в промоторы которых попадают PQS на плюс-стренде, представлен в `./files/pqs-plus-promoted.txt`  
Список генов на минус-стренде, в промоторы которых попадают PQS на минус-стренде, представлен в `./files/pqs-minus-promoted.txt`  
Общий список генов, в промоторы которых попадают квадруплексы, представлен в `./files/pqs-united-promoted.txt`

6\. На 10 баллов – сделать Gene Enrichment Analysis – зайти на сайт stringDB https://string-db.org/. Нажать на Search. Выбрать слева Multiple Proteins - Найти нужный организм, в окно загрузить список генов и нажать Search. На вкладе Analysis будут результаты обогащения отобранных генов биологическими процессами. Сделать скриншот анализа. Выгрузить таблицу в Excel. 

### Общий список генов, в промоторы которых попадают квадруплексы
![PQS GESA Analysis](./pics/pqs-gesa-analysis.png)  
см. `./files/pqs-gesa-analysis.tsv`

### Список генов, в промоторы которых попадает Z-ДНК
![Z-DNA GESA Analysis](./pics/z-gesa-analysis.png)  
см. `./files/z-gesa-analysis.tsv`